In [1]:
from numba import cuda, njit, vectorize, guvectorize, prange, float64
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.signal as signal

# load some data

In [2]:
data = loadmat('./data/sim_single_cyst.mat')
print(data.keys())

fs = np.squeeze(data['fs']) # sampling frequency
fs = np.float64(fs)
c = np.squeeze(data['c']) # speed of sound
# when c is uint16 it raise error, thus it should be casted into proper type
c = np.float64(c)

pitch = np.squeeze(data['pitch']) # pitch
angles = np.squeeze(data['angles']) # transmit angles

rf = np.squeeze(data['rf']) # radio-frequency (rf) data acquired by probe elements
rf = np.transpose(rf,(2,1,0))
# decimation - the fs=1e8 is not necessary high
dec = 4
rf = signal.decimate(rf, dec)
fs = fs/dec

print(f'sampling frequency [MHz]: {fs*1e-6}')


dict_keys(['__header__', '__version__', '__globals__', 'angles', 'c', 'fc', 'fs', 'pitch', 'rf', 'rfrc'])
sampling frequency [MHz]: 25.0


***
# FIR filter
***
<font size="4">
The finite impulse response (FIR) filter is given by equation \eqref{fir}

\begin{equation}
    y[n] = \sum_{k=0}^{N-1} h[k]x[n-k] 
    \label{fir} \tag{1}
\end{equation}

where *x* is an input signal, *h* is a filter impulse response and *y* is an output signal.
Filtration is realised by convolving the signal with impulse response of the filter.

  
    
***    
</font>

# FIR filter implementation - 1D (direct approach)

### define some functions

In [36]:
def present_signal(sig, fs, color='b', title=''):
    '''
    The function display input signal and its spectrum.
    '''
    f, p = signal.periodogram(sig,fs=fs)
    
    fig,ax = plt.subplots(nrows=1, ncols=2)
    fig.suptitle(title)
    ax[0].plot(sig,color=color)
    ax[0].set_title('signal')
    ax[0].set_xlabel('samples')
    ax[0].set_ylabel('amplitude [a.u.]')
    
    ax[1].semilogy(f*1e-6, p/np.max(psig),color=color)
    ax[1].set_title('power spectrum')
    ax[1].set_xlabel('frequency [MHz]')
    ax[1].set_ylabel('amplitude [a.u.]')
    
    plt.tight_layout()
    
    
def convolve(sig, h):
    '''
    The function convolve signals (sig) with filter impulse response (h).
    It convolve along the last dimension of sig array. 
    '''
    
    s = sig.shape
    sfshape = s[0:-1] + (s[-1] - h.shape[-1],)
    sigflt = np.zeros(sfshape)
    
    for i in range(sigflt.shape[-1]):
        sigflt[..., i] = np.sum(sig[...,i:i + h.size]*h[::-1])
        
    return sigflt

# convolve njit version
convolve_jit = njit()(convolve)


# experiment
@guvectorize([(float64[:], float64[:], float64[:])], '(),()->()')
def convolve_guv(sig, h, sigflt):
    '''
    The function convolve two signals.
    '''
#     sigflt = np.zeros(sig.size - h.size)
    for i in range(sigflt.size):
        sigflt[i] = np.sum(sig[i:i+h.size]*h[::-1])

        
@cuda.jit(device=True)
def dotprod(a, b):
    '''
    Function enumerate dot product of two vectors.
    '''
    c = 0
    for i in range(a.size):
        c += a[i]*b[i]
    return c
    
# kernel    
@cuda.jit
def firkrn(rfflt, rf, h):
    """
    Kernel function.
    Realize FIR filtration by convolving signal sig with filter impulse response h.
    
    """
    
    


    i, j, k = cuda.grid(3)
    if i > rfflt.shape[0] or j > rfflt.shape[1] or k > rfflt.shape[2]:
        return
    
    # kernel does not return a value - it modifies one of arguments into result
#     sigflt[i,j,k] = dotprod(sig[i,j,k:k+h.size], h)
    c = h
    rfflt[i,j, k]= dotprod(rf[i,j,k:k+h.size], h)
    

def fir(rf, h):
    '''
    Docstring
    '''
    
    
    n_transmissions, n_channels, n_samples = rf.shape    
    
    # allocate output array on the device
    n0, n1, n2 = rf.shape

    # put data arrays to the device
    rfflt = cuda.device_array((n0,n1,n2))
#     rfflt = np.zeros((n0,n1,n2))
#     rfflt = np.ascontiguousarray(rfflt)    
    
    
    rf = np.ascontiguousarray(rf)
    rf = cuda.to_device(rf)
    
    h = np.ascontiguousarray(h)
    h = cuda.to_device(h)

    
    
    # define thread per block and blocks per grid size
    # Note: total number of threads in block should not be bigger than 1024/512 (hardware dependent)
    blockshape = (8,8,8)
    gridshape_0 = (n0 + blockshape[0] - 1) // blockshape[0]
    gridshape_1 = (n1 + blockshape[1] - 1) // blockshape[1]
    gridshape_2 = (n2 + blockshape[2] - 1) // blockshape[2]
    gridshape = (gridshape_0, gridshape_1, gridshape_2)
    
    # run kernel
#     with cuda.defer_cleanup():
    firkrn[gridshape,blockshape](rfflt, rf, h)
    
    # transfer data from device to host
    rfflt = rfflt.copy_to_host()
    return rfflt    
    
    

In [37]:
# get some signal from rf array
sig = rf[0,0,:]

# create filter impulse response (filter coefficient array)
ntaps = 256
h = np.concatenate((np.linspace(0, 1, ntaps//2),
                    np.linspace(1, 0, ntaps//2)))

# ordinary convolve
%time sigflt = convolve(sig, h)

# convolve compiled by jit (the time gain will be visible after second run)
%time sigflt = convolve_jit(sig, h)

sigflt = np.zeros(sig.shape)
%time sigflt = fir(rf, h)


# sigflt2 = np.zeros(sig.shape)
# print(sigflt2.shape)
# convolve_guv(sig, h, sigflt2)







# # do some graphs
# fig,ax = plt.subplots(nrows=1, ncols=1)
# ax.plot(h)
# ax.set_title('filter impulse response')
# present_signal(sig, fs, 'blue', 'input signal')
# present_signal(sigflt, fs, 'red', 'filtered signal')


Exception ignored in: <finalize object at 0x7f1f46361080; dead>
Traceback (most recent call last):
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/weakref.py", line 566, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/site-packages/numba/cuda/cudadrv/driver.py", line 1804, in deref
    mem.free()
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/site-packages/numba/cuda/cudadrv/driver.py", line 1617, in free
    self._finalizer()
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/weakref.py", line 566, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/site-packages/numba/cuda/cudadrv/driver.py", line 1285, in core
    deallocations.add_item(driver.cuMemFree, handle, size)
  File "/home/zklim/bin/miniconda3/envs/ius/lib/python3.8/site-packages/numba/cuda/cudadrv/driver.py", line 970, in add_item
    self.clear(

CPU times: user 12.4 ms, sys: 122 µs, total: 12.5 ms
Wall time: 12.4 ms
CPU times: user 312 ms, sys: 17.7 ms, total: 330 ms
Wall time: 338 ms


CudaAPIError: [700] Call to cuMemAlloc results in UNKNOWN_CUDA_ERROR

In [8]:
sr = rf.shape
len(sr)

rf[0,0,0:0+3]


array([ 1.06342277e-67,  3.22474099e-68, -2.15842406e-67])